### Nonlinear regression

***This example is obtained from Kenneth J. Beers. Numerical methods for chemical engineering. Cambridge University Press, Cambridge, UK \[u.a.\], repr. edition, 2009. Chapter 8 Bayesian statistics and parameter estimation.***

We consider a batch reactor hosting a reaction $ \nu_A A+ \nu_B B -> \nu_C C$. We assume the balances volume, i.e., the reactor volume, to be isothermal ($T$=const).

The volumetric reaction rate expression is thus $r_{R1}=k_1 c_A^{\nu_A} c_B^{\nu_B}$. The initial concentrations of components $A$ and $B$ are known a priori.

We are interested in determining the stoichiometric coefficients $\nu_A$ and $\nu_B$, as well as $k_1(T)=k_1$.

For this, we exploit the relation

$\frac{dc_C}{dt}|_{t=0}=r_{R1}=k_1 [c_A(0)]^{\nu_A} [c_B(0)]^{\nu_B}$.

We run experiments with known initial concentrations of $A$ and $B$, and measure the slope of $c_C(t=0)$ and thus obtain measurements of $r_{R1}$. The measurements are given below:

In [ ]:
import numpy as np
import numpy.typing as npt

In [ ]:
c_A_measurements = np.array([0.1, 0.2, 0.1, 0.2, 0.05, 0.2])
c_B_measurements = np.array([0.1, 0.1, 0.2, 0.2, 0.2, 0.05])
r_R1_measurements = np.array([0.0246, 0.0483, 0.0501, 0.1003, 0.0239, 0.0262])*1E-03

We want to find $k_1, \nu_A, \nu_B$ using nonlinear regression.

$\frac{dc_C}{dt}|_{t=0}=r_{R1}=k_1 [c_A(0)]^{\nu_A} [c_B(0)]^{\nu_B}$.

We consider the inputs of our nonlinear regression problem to be $\boldsymbol{\tilde{x}}=[c_A(0), c_B(0)]$. The output is $\tilde{y}=r_{R1}$. The set of parameters is $\boldsymbol{\tilde{\theta}}=[k_1, \nu_A, \nu_B]$ such that we can derive

$y=\tilde{\theta}_0x_1^{\tilde{\theta}_1}x_2^{\tilde{\theta}_2}$.

In [ ]:
def rate_expression(x:npt.NDArray, k1:float, nuA:float, nuB:float) -> npt.NDArray:
    """
    Function to express the reaction rate for an irreversible reaction with reagents A and B 
    with reaction rate constant k1: A+B ->[k1] ...
    Parameters
    ----------
        x (npt.NDArray): initial concentration measurements for species A and B
        k1 (float): reaction rate constant
        nuA (float): stoichiometric coefficient of species A
        nuB (float): stoichiometric coefficient of species B
    Returns
    -------
        (npt.NDArray): reaction rate
    """

    return k1*x[0]**nuA*x[1]**nuB


Now, let's apply scipy.optimize.curve_fit!

NOTE: To use [scipy.optimize.curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html), the model function `f(x,*params)` must take the independent variable `x` as the first argument and the parameters to fit as separate remaining arguments!

In [ ]:
from scipy.optimize import curve_fit

popt, pcov = curve_fit(
    f=rate_expression, xdata=[c_A_measurements, c_B_measurements], ydata=r_R1_measurements
    )
k_1, v_A, v_B = popt
print('k_1: ', k_1)
print('v_A: ', v_A)
print('v_B: ', v_B)

In [ ]:
# Interactive 3D plot with Plotly
import plotly.graph_objects as go
c_A_range = np.linspace(0, 0.25, 50)
c_B_range = np.linspace(0, 0.25, 50)
C_A, C_B = np.meshgrid(c_A_range, c_B_range)
R_R1_surface = rate_expression(
    x=[C_A, C_B], k1=k_1, nuA=v_A, nuB=v_B
    )
# Create the surface
fig = go.Figure(data=[
    go.Surface(
        x=c_A_range,
        y=c_B_range,
        z=R_R1_surface,
        colorscale='Viridis',
        opacity=0.7,
        name='Fitted Surface'
    ),
    # Add scatter points
    go.Scatter3d(
        x=c_A_measurements,
        y=c_B_measurements,
        z=r_R1_measurements,
        mode='markers',
        marker=dict(
            size=8,
            color=r_R1_measurements,
            colorscale='Reds',
            showscale=False,
            line=dict(color='black', width=2)
        ),
        name='Data Points'
    )
])

# Update layout
fig.update_layout(
    title='Nonlinear Regression: 3D Surface Fit',
    scene=dict(
        xaxis_title='Concentration c_A (mol/L)',
        yaxis_title='Concentration c_B (mol/L)',
        zaxis_title='Reaction Rate (mol/L/s)'
    ),
    width=900,
    height=700
)

fig.show()